In [1]:
# -*- coding:utf-8-*-
import pandas as pd
import string, re, collections
from collections import namedtuple
import numpy as np

# check if the feature includes words in our words_library

def ads_search(obj_content):
    if obj_content:
        file_obj = open('/home/wilson/Desktop/databases/words_library.txt', 'r')
        words_library = file_obj.read()
        ads_library = re.compile(words_library, re.I)
#         print(obj_content)
        result = ads_library.findall(obj_content)
        if result:
            
            return 0 # exist ads return 0
        else:
            return 1
    else:
        return 1


# check if the feature includes phone numbers
def phone_search(feature_s):
    if feature_s:
        phone_re = re.compile('(\d{3}\D{0,1}\d{3}\D{0,1}\d{4})')
        phone_label = phone_re.findall(feature_s)
        if phone_label:
            return 0# contains 0
        else:
            #         print("This sentence doesn't contain phone num")
            return 1
    else:
        return 1


# count the length of a string
def string_len(sentence_in):
    count_en = count_dg = count_sp = count_zh = count_pu = 0
    if sentence_in:
        s_leng = len(sentence_in)
        for one in sentence_in:
            if one in string.ascii_letters:
                count_en += 1
            elif one.isdigit():
                count_dg += 1
            elif one.isspace():
                count_sp += 1
            elif one.isalpha():
                count_zh += 1
            else:
                count_pu += 1

        total_chars = count_en + count_dg + count_sp + count_zh + count_pu
        if total_chars == s_leng:
            return (count_en, count_dg, count_sp, count_zh, count_pu)
        else:
            print('String length counting is wrong')
            return None
    else:
        return (count_en, count_dg, count_sp, count_zh, count_pu)


# count the num of english words in one signature
def count_words(sentence_in):
    if sentence_in:
        word_find = re.compile('[a-zA-Z0-9]+')
        counter = collections.Counter(word_find.findall(sentence_in))
        kind_words = len(counter)
        num_words = sum(counter.values())
        return (kind_words, num_words)
    else:
        return(0,0)

def apply_and_concat(dataframe, field, func, column_names):
    return pd.concat((
        dataframe,
        dataframe[field].apply(
            lambda cell: pd.Series(func(cell), index=column_names))), axis=1)



if __name__ == "__main__":

#     df_name_sig = pd.read_csv('/home/wilson/Desktop/databases/content_sig_labeling.csv')
    df_name_sig = pd.read_csv('/home/wilson/Desktop/databases/content_sig_labeling_withNaN.csv')

    df_name_sig.drop_duplicates('user_nickname',inplace=True)
    df_name_sig['signature'] = df_name_sig['signature'].fillna(0)
    df_string = apply_and_concat(df_name_sig, 'signature', string_len, ['count_en', 'count_dg', 'count_sp', 'count_zh', 'count_pu'])
    df_string['ads_status'] = df_name_sig['signature'].apply(ads_search)
    df_string['phone_status'] = df_name_sig['signature'].apply(phone_search)
    df_string['phone_status'] = df_name_sig['signature'].apply(phone_search)
    df_string = apply_and_concat(df_string, 'signature', count_words, ['kind_words', 'num_words'])
    
    df_string['name_status'] = df_name_sig['user_nickname'].apply(ads_search)


In [2]:
df_string.to_csv('/home/wilson/Desktop/databases/signature_scalar.csv', header=True)

In [3]:
df_update = pd.read_csv('/home/wilson/Desktop/databases/signature_scalar.csv',index_col=0)

In [4]:
df_string.head()

,user_nickname,content,signature,count_en,count_dg,count_sp,count_zh,count_pu,ads_status,phone_status,kind_words,num_words,name_status
0,巫教练,恭喜恭喜！,明通驾校一温莎首家华人驾校\n电话:519—257—8840,0,10,1,15,3,0,0,3,3,0
1,YiF,哦,YiF，没有如果,3,0,0,4,1,1,1,1,1,1
2,Jim,面对罪犯挥舞手中的手枪，那个警员冒着危险，冷静处理这事，抓住了罪犯，值得称赞👍 - 这种情况...,最后让冰溶解的一定是春天,0,0,0,12,0,1,1,0,0,1
3,塬,周教练教的好[呲牙],0,0,0,0,0,0,1,1,0,0,1
4,銘銘,提供明天中午去多伦多的carpool,0,0,0,0,0,0,1,1,0,0,1


In [5]:
# sometimes the user_nickname is empty, it cannot be converted into numeric variables
df_string[df_string['user_nickname'].isnull()]

,user_nickname,content,signature,count_en,count_dg,count_sp,count_zh,count_pu,ads_status,phone_status,kind_words,num_words,name_status
